In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rawTrainData = pd.read_csv("./train.csv")
rawTestX = pd.read_csv("./test.csv")

rawTrainX = rawTrainData.drop(['SalePrice'], axis=1);
trainY = rawTrainData['SalePrice']

rawFull = rawTrainX.append(rawTestX, ignore_index = True )

In [30]:
def replace_empties(df):
    change_to_none = ['Alley',
    'BsmtCond','BsmtQual','BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PoolQC',
    'Fence',
    'MiscFeature']
    
    change_to_0 = ['BsmtFullBath', 'BsmtHalfBath', 'TotalBsmtSF',
     'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
     'GarageArea', 'GarageCars'];
 
    df[change_to_none] = df[change_to_none].fillna('None')
    df[change_to_0] = df[change_to_0].fillna(0)
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df.YearBuilt)
    
    stringTypes = df.select_dtypes(include = [object]).columns
    numericTypes = df.select_dtypes(exclude=['object']).columns

    df[stringTypes] = df[stringTypes].fillna(df[stringTypes].median())
    df[numericTypes] = df[numericTypes].fillna(df[numericTypes].mean())

    return df

In [31]:
def get_dummies(df):
    stringTypes = df.select_dtypes(include = [object]).columns
    dummies = pd.get_dummies(df[stringTypes])
    df = df.drop(columns=stringTypes);
    df = df.join(dummies)
    return df

In [32]:
def normalize_features(df):
    df.iloc[:,1:]=(df.iloc[:,1:]-df.iloc[:,1:].mean())/df.iloc[:,1:].std()
    return df

In [33]:
def transform_features(df):
    df = replace_empties(df)
    df = get_dummies(df)
    df = normalize_features(df)
    return df

In [49]:
def train(X, y, iterations = 10000, alfa = 0.005):
    w = np.random.rand(X.shape[1])
    b = 0.5
    
    m = X.shape[0]
    
    x_numpy = X.values
    y_numpy = y.values
    for i in range(0, iterations): 
        pred_y = (np.dot(x_numpy, w) + b)
        w = w - (alfa/m * np.dot(x_numpy.T, pred_y - y_numpy))
        b = b - (alfa/m * np.sum(pred_y - y_numpy)) 
    #pred_y = (np.dot(x_numpy, w) + b)
    #J = np.sum((pred_y - y_numpy) * (pred_y - y_numpy)) / m
    #print(J)
    return w, b

In [50]:
newX = transform_features(rawFull)
newTrainX = newX.iloc[0:rawTrainData.shape[0]].drop(['Id'], axis=1);
w, b = train(newTrainX, trainY)


423254564.652


In [43]:
out = open('output_1.csv', "w")
out.write("Id,SalePrice\n")

newTestX = newX.iloc[rawTrainData.shape[0]:]
x_test_numpy = newTestX.drop(['Id'], axis=1).values
res = np.dot(x_test_numpy, w) + b

rows = [''] * newTestX.shape[0] # predefine or use append
for num in range(0, newTestX.shape[0]):
    label = max(res[num], 0)
    id = newTestX['Id'].iloc[num]
    rows[num] = "%d,%f\n"%(id,label)

out.writelines(rows)
out.close()